# AMAL - TP 2 - Graphes de calcul, autograd et modules

L'objectif de ce TP est d'automatiser les fonctions codées à la main lors du TP 1. On le fait en 3 étapes :
+ 1. Calcul automatique du gradient par autograd
+ 2. Descente de gradient automatique avec la classe `torch.optim`
+ 3. Modules préprogrammés et conteneurs

Dans ce notebook, on implémente le point 3. La partie intéressante se situe dans la fonction `run_gradient_descent`, en particulier les lignes suivantes

```python
mse = torch.nn.MSELoss(reduction='mean')
    
model = torch.nn.Sequential(
      torch.nn.Linear(n_attribute, nb_neurons),
      torch.nn.Tanh(),
      torch.nn.Linear(nb_neurons, 1)
    )
```

In [1]:
import numpy as np
import torch
from torch.autograd import Function
from torch.autograd import gradcheck
from datamaestro import prepare_dataset 
from torch.utils.tensorboard import SummaryWriter
import time

## Implémentation de l'algorithme de descente de gradient

La fonction `run_gradient_descent` permet d'implémenter les trois variantes de descente de gradient :
+ Gradient batch: `n_batch` = 1
+ Gradient stochastique: `n_batch` = N où N est le nombre de lignes de données
+ Gradient mini-batch: 1 < `n_batch` < N

In [2]:
def run_gradient_descent(data, nb_iteration, n_batch, nb_neurons, write=False):
    
    N = data.shape[0]   # nombre de données
    n_attribute = data.shape[1]-1
    
    if write:
        writer = SummaryWriter(log_dir='runs/nb_batch=' + str(n_batch) + 'nb_pass=' + str(nb_iteration/n_batch) + '_timestamp=' + str(int(time.time())))
    
    eps = 0.05 # Pas de la descente de gradient
    
    mse = torch.nn.MSELoss(reduction='mean')
    
    model = torch.nn.Sequential(
          torch.nn.Linear(n_attribute, nb_neurons),
          torch.nn.Tanh(),
          torch.nn.Linear(nb_neurons, 1)
        )
    
    optim = torch.optim.SGD(params=model.parameters(),lr=eps)
    optim.zero_grad()

    L_train = 0     # perte pour la partie train
    for k in range(nb_iteration):
        
        # Choix des lignes des données qui servent pour cette itération
        if n_batch == N: # stochastic 
            idx = [np.random.randint(0, N//2)]
        else:   # batch or mini-batch
            idx_min = int(N/n_batch * k) % N
            idx_max = int(N/n_batch * (k+1)) % N
            if idx_min < idx_max:
                idx = range(idx_min, idx_max)
            else:
                idx = list(range(idx_min, N)) + list(range(0, idx_max))
            
        X = torch.from_numpy(data[idx, :-1])
        y = torch.from_numpy(data[idx][: , [-1]])
        loss = mse(model(X.float()), y.float())
        L_train = (k*L_train + loss) / (k+1)    # update L_train
        loss.backward()
        optim.step()
        optim.zero_grad()
    
        if k % 10 == 0 and write:
            # Pour comparer les différente méthodes, on compte le nombre de lignes de données utilisées à chaque itération
            writer.add_scalar('Loss/train', L_train/len(idx), k*len(idx))

    with torch.no_grad():
        L_test = 0  # perte pour la partie test
        for k in range(int(N/2)):     # on teste sur la seconde moitié du data set
            x = torch.from_numpy(data[[int(N/2) + k], :-1])
            y = torch.from_numpy(data[[int(N/2) + k]][: , [-1]])
            loss = mse(model(x.float()), y.float())
            L_test = (k*L_test + loss) / (k+1)
            if write:
                writer.add_scalar('Loss/test', L_test, k)
    
    print('Test loss: ' + str(L_test.item()))
    if write:
        writer.close()

# Test avec les données de Boston Housing

In [3]:
## Pour telecharger le dataset Boston
ds=prepare_dataset("edu.uci.boston")
fields, data =ds.files.data() 
N = data.shape[0]
n_attribute = data.shape[1]-1

In [4]:
# Pour randomizer les données et diminuer le conditionnement
np.random.shuffle(data)
data = data / data.max(axis=0)

In [5]:
# On compare les trois variantes (batch / stochastic / mini-batch) en utilisant à chaque fois au total `n_pass` fois les données
# Le nombre d'itérations n'est donc pas le même selon les variantes
n_pass = 100
nb_neurons = 10
run_gradient_descent(data, n_pass, 1, nb_neurons, write=True)    # batch
run_gradient_descent(data, n_pass*N, N, nb_neurons, write=True)  # stochastic
n_batch = 10
run_gradient_descent(data, n_pass*n_batch, n_batch, nb_neurons, write=True)   # mini-batch

Test loss: 0.02113339677453041
Test loss: 0.006567567121237516
Test loss: 0.010460889898240566
